In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
print("TensorFlow version:", tf.__version__)
# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime
import shutil

TensorFlow version: 2.16.1


In [2]:
 %reload_ext tensorboard
from tensorflow.keras import Model
from pathlib import Path
import pandas as pd

In [3]:
BATCH_SIZE = 64
BUFFER_SIZE = 128
LEARNING_RATE = 0.05
EPOCHS = 50

In [4]:
input_dir = Path('./data/prepared')
logs_path = Path('./data/logs')
model_path = Path('./data/models/nn')
if logs_path.exists():
  shutil.rmtree(logs_path)
logs_path.mkdir(parents=True)

X_train_name = input_dir / 'X_train.csv'
y_train_name = input_dir / 'y_train.csv'
X_test_name = input_dir / 'X_test.csv'
y_test_name = input_dir / 'y_test.csv'

X_train = pd.read_csv(X_train_name)
y_train = pd.read_csv(y_train_name)
X_test = pd.read_csv(X_test_name)
y_test = pd.read_csv(y_test_name)

train_ds = tf.data.Dataset.from_tensor_slices(
    (X_train, y_train)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)

In [5]:
X_train.shape

(460, 8)

In [6]:
train_ds

<_BatchDataset element_spec=(TensorSpec(shape=(None, 8), dtype=tf.int64, name=None), TensorSpec(shape=(None, 1), dtype=tf.float64, name=None))>

In [9]:
@tf.keras.utils.register_keras_serializable()
class SomeModel(Model):
  def __init__(self, neurons_cnt=128):
    super(SomeModel, self).__init__()

    self.d_in = Dense(8, activation='relu')
    self.d1 = Dense(neurons_cnt, activation='relu')
    self.d2 = Dense(neurons_cnt, activation='relu')
    self.d_out = Dense(1)

  def get_config(self):
        base_config = super().get_config()
        config = {
            "SomeModel": keras.saving.serialize_keras_object(self.SomeModel),
        }
        return {**base_config, **config}
      
  @classmethod
  def from_config(cls, config):
        sublayer_config = config.pop("sublayer")
        sublayer = keras.saving.deserialize_keras_object(sublayer_config)
        return cls(sublayer, **config)

  def call(self, x):
    x = self.d_in(x)
    x = self.d1(x)
    return self.d_out(x)

In [10]:
# Create an instance of the model
model = SomeModel()

In [11]:
loss_object = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.SGD(learning_rate=LEARNING_RATE)

In [12]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.MeanAbsoluteError(name='train_mae')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.MeanAbsoluteError(name='test_mae')

In [13]:
@tf.function
def train_step(input_vector, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(input_vector, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [14]:
@tf.function
def test_step(input_vector, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(input_vector, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [15]:
from tensorflow import keras

In [16]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = logs_path / 'gradient_tape' / current_time / 'train'
train_log_dir.mkdir(exist_ok=True, parents=True)
test_log_dir = logs_path / 'gradient_tape' / current_time / 'test'
test_log_dir.mkdir(exist_ok=True, parents=True)
train_summary_writer = tf.summary.create_file_writer(str(train_log_dir))
test_summary_writer = tf.summary.create_file_writer(str(test_log_dir))

logdir=logs_path / "fit" / datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir.mkdir(exist_ok=True, parents=True)
fit_summary_writer = tf.summary.create_file_writer(str(logdir))

In [17]:
tf.summary.trace_on(graph=True, profiler=True, profiler_outdir=str(logdir))
for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  for (x_train, y_train) in train_ds:

    with fit_summary_writer.as_default():
      train_step(x_train, y_train)


  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (x_test, y_test) in test_ds:
    test_step(x_test, y_test)

  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('mae', test_accuracy.result(), step=epoch)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test MAE: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result(),
                         test_loss.result(),
                         test_accuracy.result()))

  # Reset metrics every epoch
  train_loss.reset_state()
  test_loss.reset_state()
  train_accuracy.reset_state()
  test_accuracy.reset_state()

with fit_summary_writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0
  )

C:\Users\ru-lo\PycharmProjects\AICourseTGroup\venv\Lib\site-packages\keras\src\layers\layer.py:360: UserWarning: `build()` was called on layer 'some_model', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Epoch 1, Loss: 259.7462158203125, Accuracy: 12.502402305603027, Test Loss: 7.229612350463867, Test MAE: 2.5689401626586914
Epoch 2, Loss: 9.607307434082031, Accuracy: 2.976243495941162, Test Loss: 14.93819808959961, Test MAE: 3.7688310146331787
Epoch 3, Loss: 7.211973667144775, Accuracy: 2.569486141204834, Test Loss: 7.22495174407959, Test MAE: 2.5766377449035645
Epoch 4, Loss: 5.403367042541504, Accuracy: 2.2597506046295166, Test Loss: 3.121629238128662, Test MAE: 1.658613920211792
Epoch 5, Loss: 3.820139169692993, Accuracy: 1.8106433153152466, Test Loss: 4.619668006896973, Test MAE: 2.037017583847046
Epoch 6, Loss: 2.462649345397949, Accuracy: 1.4157978296279907, Test Loss: 2.465289354324341, Test MAE: 1.463378667831421
Epoch 7, Loss: 2.7044942378997803, Accuracy: 1.5163865089416504, Test Loss: 3.9214446544647217, Test MAE: 1.87583327293396
Epoch 8, Loss: 2.986365556716919, Accuracy: 1.5995314121246338, Test Loss: 3.4639806747436523, Test MAE: 1.7547669410705566
Epoch 9, Loss: 1.9090

In [18]:
model.save(model_path / 'mymodel.keras')

In [19]:
loaded_model = tf.keras.models.load_model(model_path / 'mymodel.keras')

TypeError: <class '__main__.SomeModel'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': None, 'class_name': 'SomeModel', 'config': {'trainable': True, 'dtype': 'float32'}, 'registered_name': 'Custom>SomeModel', 'build_config': {'input_shape': [64, 8]}}.

Exception encountered: Unable to revive model from config. When overriding the `get_config()` method, make sure that the returned config contains all items used as arguments in the  constructor to <class '__main__.SomeModel'>, which is the default behavior. You can override this default behavior by defining a `from_config(cls, config)` class method to specify how to create an instance of SomeModel from its config.

Received config={'trainable': True, 'dtype': 'float32'}

Error encountered during deserialization: SomeModel.__init__() got an unexpected k

In [ ]:
loaded_model

In [ ]:
model

In [ ]:
# Let's check:
import numpy as np
np.testing.assert_allclose(
    model.predict(X_test),
    loaded_model.predict(X_test)
)
%tensorboard --logdir ./data/logs

In [ ]:
%tensorboard --logdir ./data/logs/gradient_tape